In [1]:
# 문장 텍스트와 질문을 모델에 입력하고 그에 대한 답변을 하도록 함
# 기계 톡해 이해(machine reading comprehension, MRC)의 하위 카테고리로 분류 가능
# 답변 과정은 (주어진 context에서)추출(Extractive)과 생성(Generate)(context참고하여 새로 작성)의 두 가지로 나뉜다
# 인코더형은 추출에, 디코더형은 생성에 강한 편임

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = 'klue/bert-base'  #인코더 기반 모델임
tokenizer = AutoTokenizer.from_pretrained(model_name)
# 모델에 레이블 개수 지정하지 않음. 기본적으로 추출 방식으로, 답변에 해당하는 부분의 시작과 끝 인덱스를 출겨하는 방식이기 때문임.
model = AutoModelForQuestionAnswering.from_pretrained(model_name) 

model # 맨 끝에 qa_outputs 레이어 있음

2025-06-18 04:58:17.313907: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-18 04:58:17.320903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750222697.329130   40800 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750222697.331587   40800 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750222697.337952   40800 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [3]:
from datasets import load_dataset

dataset = load_dataset("klue", "mrc")
sample = dataset['train'][0]

print(f"내용: {sample['context'][:50]}")
print(f"질문: {sample['question']}")
print(f"답변: {sample['answers']}")

train-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

내용: 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 
질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
답변: {'answer_start': [478, 478], 'text': ['한 달가량', '한 달']}


In [9]:
# 학습 전 전처리 
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]  # 문장 앞뒤 공백 제거
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",    # 일부 데이터는 최대 입력 길이를 초과할 수 있으므로 두번째 문장에 대해서만 잘라내도록 처리
        return_offsets_mapping=True, # 인코딩된 토큰의 원문장내 위치를 알 수 있도록 인덱스 반환 
        padding="max_length"
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions=[]
    end_positions=[]

    #답변의 시작과 끝을 원 context에 매핑
    for i,offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i) # 문장내 context의 인덱스 범위를 확인

        # Context 시작,끝 찾기
        idx = 0
        while sequence_ids[idx] !=1 :
            idx+=1
        context_start = idx
        while sequence_ids[idx] ==1 :
            idx+=1
        context_end = idx -1

        # 답변이 context 내에 포함되지 않으면 레이블 (0,0)으로 지정
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx<=context_end and offset[idx][0]<=start_char:
                idx+=1
            start_positions.append(idx-1)

            idx = context_end
            while idx>=context_start and offset[idx][1]>=end_char:
                idx-=1
            end_positions.append(idx+1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs



tokenized_dataset = dataset.map(
    preprocess_function, 
    batched=True,
    remove_columns=dataset['train'].column_names
)



Map:   0%|          | 0/17554 [00:00<?, ? examples/s]

Map:   0%|          | 0/5841 [00:00<?, ? examples/s]

In [12]:
# 본래는 padding 처리 해야하지만, 이 경우 데이터셋이 이미 padding 처리가 되어 있음
# 이에 데이터 타입과 차원만 맞춰주는 DefaultDataCollator 사용
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()
batch = data_collator([tokenized_dataset['train'][i] for i in range(10)])

batch

{'input_ids': tensor([[    2,  1174, 18956,  ...,  2170,  2259,     3],
         [    2,  3920, 31221,  ...,  8055,  2867,     3],
         [    2,  8813,  2444,  ...,  3691,  4538,     3],
         ...,
         [    2,  6860, 19364,  ...,  2532,  6370,     3],
         [    2, 27463, 23413,  ..., 21786,  2069,     3],
         [    2,  3659,  2170,  ...,  2470,  3703,     3]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         ...,
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]),
 'start_positions': tensor([260,  31,   0,  80,  72,  81, 216, 348, 323, 348]),
 'end_positions': tensor([263,  33,   0,  81,  78

In [13]:
# 이전에 불러온 klue/bert-base 모델 활용
with torch.no_grad():
    outputs=model(**batch)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = batch['input_ids'][0,answer_start_index:answer_end_index+1]
tokenizer.decode(predict_answer_tokens)

''

In [ ]:
# evaluate.load("squad")로 질의응답 평가 진행이 가능하지만, 후처리와 시간이 너무 많이 소요하므로 learnign loss로 대체